<a href="https://colab.research.google.com/github/arazshah/Arz-File/blob/master/GWLA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**developed to predict GWL changes**

In [27]:
import ee
import geemap
import geopandas as gpd
import pandas as pd
import tensorflow
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [40]:
ee.Authenticate()
ee.Initialize(project='arazshah')

In [41]:
# define time period
start_date='2002-01-01'
end_date='2002-06-30'

In [42]:
#define ROI
shape_file='shp/urmia_lake_basin.shp'
gdf=gpd.read_file(shape_file)
polygon=gdf.geometry.iloc[0]
minX,minY,maxX,maxY=polygon.bounds
roi=ee.Geometry.Rectangle([minX,minY,maxX,maxY])

In [43]:
# precipitation DataSet
precipitation = "UCSB-CHG/CHIRPS/PENTAD"

In [53]:
# minimume temperature DataSet
min_temp="IDAHO_EPSCOR/TERRACLIMATE"

In [54]:
# maximum temperature DataSet
max_temp="IDAHO_EPSCOR/TERRACLIMATE"

In [60]:
# EvapoTranspiration DataSet
evapotranspiration="MODIS/006/MOD16A2"

In [56]:
#GRACE GWS DataSet
grace="NASA/GRACE/MASS_GRIDS/LAND"

In [45]:
def extract_monthly_time_series(start_date, end_date,dataset,band_name,roi):
  image = ee.ImageCollection(dataset) \
  .filterDate(start_date, end_date) \
  .select(band_name) \
  .map(lambda image: image.clip(roi))
  def image_to_feature(img):
     # Reduce the image over the region of interest
    reduced = img.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=roi,
        scale=1000
    )
    value=reduced.get(band_name)
    date=img.date().format('YYYY-MM-dd')
    return ee.Feature(None,{'date':date,'band':value})


  features= image.map(image_to_feature)
  feature_collection=ee.FeatureCollection(features)
  feature_list=feature_collection.getInfo()['features']

  data=[
      {
          'date':feature['properties']['date'],
          'band':feature['properties']['band']
      }
      for feature in feature_list
  ]
  df=pd.DataFrame(data)

  df['date']=pd.to_datetime(df['date'])
  df.set_index('date',inplace=True)
  df.sort_index(inplace=True)
  return df

In [39]:
precipitation_df=extract_monthly_time_series(start_date,end_date,precipitation,'precipitation',roi)

KeyboardInterrupt: 

In [57]:
min_temp_df=extract_monthly_time_series(start_date,end_date,min_temp,'tmmn',roi)

EEException: Error in map(ID=200201):
Image.reduceRegion: Too many pixels in the region. Found 88105405717, but maxPixels allows only 10000000.
Ensure that you are not aggregating at a higher resolution than you intended; that is a frequent cause of this error. If not, then you may set the 'maxPixels' argument to a limit suitable for your computation; set 'bestEffort' to true to aggregate at whatever scale results in 'maxPixels' total pixels; or both.

In [58]:
max_temp_df=extract_monthly_time_series(start_date,end_date,max_temp,'tmmx',roi)

EEException: Error in map(ID=200201):
Image.reduceRegion: Too many pixels in the region. Found 88105405717, but maxPixels allows only 10000000.
Ensure that you are not aggregating at a higher resolution than you intended; that is a frequent cause of this error. If not, then you may set the 'maxPixels' argument to a limit suitable for your computation; set 'bestEffort' to true to aggregate at whatever scale results in 'maxPixels' total pixels; or both.

In [61]:
evapotranspiration_df=extract_monthly_time_series(start_date,end_date,evapotranspiration,'ET',roi)

EEException: Error in map(ID=2002_01_01):
Image.reduceRegion: The 'geometry' parameter is at least partly outside the selected projection's area of validity.

In [ ]:
grace_df=extract_monthly_time_series(start_date,end_date,grace,'lwe_thickness_jpl',roi)

In [ ]:
# Merge the dataframes on the 'date' column
data=precipitation_df.merge(min_temp_df,on='date') \
  .merge(max_temp_df,on='date') \
  .merge(evapotranspiration_df,on='date') \
  .merge(grace_df,on='date')

In [ ]:
# Drop the 'date' column and handle any missing values if needed
data = data.dropna().reset_index(drop=True)
X = data.drop(columns=['lwe_thickness'])  # Input features
y = data['lwe_thickness']  # Target variable (groundwater storage changes)

In [ ]:
# Normalize the input features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
X_scaled = X_scaled.reshape((X_scaled.shape[0], X_scaled.shape[1], 1))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)